In [43]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from tensorflow.keras import applications,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import keras.backend as K

In [24]:
P = '../input/15-after-processed/15_after_processed/'

In [25]:
train_image = tf.keras.utils.image_dataset_from_directory( P+'train_images/',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),  
)

val_image = tf.keras.utils.image_dataset_from_directory(  P+ 'val_images/',
    labels="inferred",
    label_mode="categorical",
    class_names=None, 
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
)



Found 21108 files belonging to 10019 classes.
Found 5414 files belonging to 4190 classes.


In [27]:
# Load the train and validation CSV files
train_csv = pd.read_csv(P+'train.csv')
val_csv = pd.read_csv(P+'val.csv')

In [44]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
# Get the number of classes in the training set
num_classes = len(train_image.class_names)

In [45]:
# Load the ResNet50 model
resnet = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg',
)

# Freeze the layers of the ResNet50 model
for layer in resnet.layers:
    layer.trainable = False

# Add a new output layer
output = tf.keras.layers.Dense(num_classes, activation='softmax')(resnet.output)
model = tf.keras.models.Model(inputs=resnet.input, outputs=output)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=[get_f1],
)

# Define a function to preprocess the labels in the validation set
def preprocess_labels(image, label):
    label = tf.argmax(label, axis=-1)
    label = tf.one_hot(label, num_classes)
    return image, label

# Preprocess the labels in the validation set
val_image = val_image.map(preprocess_labels)

# Train the model on the preprocessed training data
history = model.fit(
    train_image,
    validation_data=val_image,
    epochs=10,
)

# Compute the accuracy of the predictions on the validation set
val_loss, val_acc = model.evaluate(val_image)
print('Validation accuracy:', val_acc)


Epoch 1/10
660/660 [==============================] - 93s 134ms/step - loss: 8.6555 - get_f1: 0.2854 - val_loss: 19.6565 - val_get_f1: 2.4510e-04
Epoch 2/10
660/660 [==============================] - 91s 137ms/step - loss: 0.4262 - get_f1: 0.9151 - val_loss: 21.7249 - val_get_f1: 2.4010e-04
Epoch 3/10
660/660 [==============================] - 90s 135ms/step - loss: 0.2447 - get_f1: 0.9614 - val_loss: 22.0594 - val_get_f1: 4.2481e-04
Epoch 4/10
660/660 [==============================] - 92s 139ms/step - loss: 0.1873 - get_f1: 0.9743 - val_loss: 22.2913 - val_get_f1: 2.2198e-04
Epoch 5/10
660/660 [==============================] - 90s 135ms/step - loss: 0.1588 - get_f1: 0.9791 - val_loss: 22.7313 - val_get_f1: 2.0640e-04
Epoch 6/10
660/660 [==============================] - 90s 135ms/step - loss: 0.1461 - get_f1: 0.9821 - val_loss: 22.9094 - val_get_f1: 2.3529e-04
Epoch 7/10
660/660 [==============================] - 92s 137ms/step - loss: 0.1497 - get_f1: 0.9801 - val_loss: 23.9067 - v